In [21]:
from urllib.request import urlopen
import json
from geojson import FeatureCollection, dump
import datetime

In [22]:
# Configure different leggers to be merged and the end and start date to set
leggers = [{
    'start_date': '01-01-2010',
    'end_date': '30-04-2020',
    'url': 'https://geoservices.rijkswaterstaat.nl/apps/geoserver/rws_vegetatielegger/ows?service=WFS&version=2.0.0&request=GetFeature&typeName=vegetatievlakken_2014&outputFormat=json&srsName=epsg:4326'  
}, {
    'start_date': '01-05-2020',
    'end_date': '31-12-2030',
    'file': 'D:/vegetatiemonitor/data/vegetatielegger_2020.geojson'
#     'url': 'https://geoservices.rijkswaterstaat.nl/apps/geoserver/rws_vegetatielegger/ows?service=WFS&version=2.0.0&request=GetFeature&typeName=vegetatieklassen&outputFormat=json' 
#     --> this jsonm does not contain the correct properties yet
}]


In [23]:
features = []
for legger in leggers: 
    if 'url' in legger:
        response = urlopen(legger['url'])
        data = json.loads(response.read())
    elif 'file' in legger: 
        with open(legger['file']) as json_file:
            data = json.load(json_file)
            
    for feature in data['features']: 
        feature['properties']['start_date'] = datetime.datetime.strptime(legger['start_date'], "%d-%m-%Y").timestamp()
        feature['properties']['end_date'] = datetime.datetime.strptime(legger['end_date'], "%d-%m-%Y").timestamp()
    features.extend(data['features'])


In [24]:
collection = FeatureCollection(features)


In [25]:
with open('vegetatievlakken-combo.geojson', 'w') as f:
    json.dump(collection, f)